# Crossmatching Scripts to enhance GBS version 3 by Soubiran et al. (2023)

## Author: Sven Buder (ANU) sven.buder@anu.edu.au

Created on 240102

In [37]:
# Preamble
import numpy as np
from astroquery.gaia import Gaia
from astropy.table import Table, vstack, join
from astropy.io import fits

In [264]:
# Login to Gaia archive: https://gea.esac.esa.int/archive/
user = 'sbuder'
Gaia.login(user = user)

INFO: Login to gaia TAP server [astroquery.gaia.core]
Password: ········
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


### 2.1) Upload initial file to Gaia archive

In [265]:
# Upload our original file
try:
    gbs_v3_original = Table.read('individual_crossmatches/gbs_v3_tmass_id_original.fits')
    Gaia.upload_table(upload_resource=gbs_v3_original, table_name='gbs_v3')
except:
    print('Could not upload table. Does gbs_v3 already exist maybe?\n')

# Upload the allwise identifier match file
try:
    gbs_v3_allwise = Table.read('individual_crossmatches/gbs_v3_allwise_id.fits')
    Gaia.upload_table(upload_resource=gbs_v3_allwise, table_name='gbs_v3_allwise')
except:
    print('Could not upload table. Does gbs_v3_allwise already exist maybe?')

Sending pytable.
500 Error 500:
esavo.tap.TAPException: Error while uploading table "gbs_v3" : problems with the input (can be due to format, inconsistencies in metadata-data...) [Can not execute the following SQL: 
CREATE TABLE user_sbuder.gbs_v3 ( "hip" character varying(9), "hip_int" bigint, "hd" character varying(8), "ra_j2000" double precision, "dec_j2000" double precision, "tmass_id" character varying(16), "theta_ld" real, "e_theta_ld" real, "ref_theta_ld" character varying(20), "teff" smallint, "e_teff" smallint, "e_teff_percent" real, "logg" real, "e_logg" real, "fe_h" real,  gbs_v3_oid SERIAL ) TABLESPACE gacs_netapp
Because: ERROR: relation "gbs_v3" already exists]
Could not upload table. Does gbs_v3 already exist maybe?

Sending pytable.
500 Error 500:
esavo.tap.TAPException: Error while uploading table "gbs_v3_allwise" : problems with the input (can be due to format, inconsistencies in metadata-data...) [Can not execute the following SQL: 
CREATE TABLE user_sbuder.gbs_v3_al

### 2.2) Crossmatch with 2MASS

In [266]:
tmass_query = """
SELECT
my_table.hip_int,
tmass.*
FROM gaiadr1.tmass_original_valid as tmass
INNER JOIN
    user_"""+user+""".gbs_v3 as my_table
    ON my_table.tmass_id = tmass.designation
"""

xmatch_tmass = Gaia.launch_job(query = tmass_query)
xmatch_tmass_results = xmatch_tmass.get_results()
# fix formatting issue for string columns to allow to write to file
for key in xmatch_tmass_results.keys():
    if xmatch_tmass_results[key].dtype == object:
        xmatch_tmass_results[key] = np.array(xmatch_tmass_results[key],dtype=str)
# export as fits file
xmatch_tmass_results.write('individual_crossmatches/gbs_v3_tmass-result.fits',overwrite=True)

In [300]:
test = xmatch_gaia_results
for key in test.keys():
    print(key,'\n',test[key].unit,'\n')

hip_int 
 None 

tmass_id 
 None 

source_id 
 None 

r_med_geo 
 pc 

r_lo_geo 
 pc 

r_hi_geo 
 pc 

r_med_photogeo 
 pc 

r_lo_photogeo 
 pc 

r_hi_photogeo 
 pc 

flag 
 None 

solution_id 
 None 

designation_gaiadr3 
 None 

source_id_2 
 None 

random_index 
 None 

ref_epoch 
 yr 

ra_gaiadr3 
 deg 

ra_error_gaiadr3 
 mas 

dec_gaiadr3 
 deg 

dec_error_gaiadr3 
 mas 

parallax 
 mas 

parallax_error 
 mas 

parallax_over_error 
 None 

pm 
 mas / yr 

pmra 
 mas / yr 

pmra_error 
 mas / yr 

pmdec 
 mas / yr 

pmdec_error 
 mas / yr 

ra_dec_corr 
 None 

ra_parallax_corr 
 None 

ra_pmra_corr 
 None 

ra_pmdec_corr 
 None 

dec_parallax_corr 
 None 

dec_pmra_corr 
 None 

dec_pmdec_corr 
 None 

parallax_pmra_corr 
 None 

parallax_pmdec_corr 
 None 

pmra_pmdec_corr 
 None 

astrometric_n_obs_al 
 None 

astrometric_n_obs_ac 
 None 

astrometric_n_good_obs_al 
 None 

astrometric_n_bad_obs_al 
 None 

astrometric_gof_al 
 None 

astrometric_chi2_al 
 None 

astrometric_ex

### 2.3) Crossmatch with Gaia DR3

In [267]:
gaia_query = """
SELECT
my_table.hip_int,
my_table.tmass_id,
calj.*,
gaia.*
FROM gaiadr3.gaia_source as gaia
LEFT OUTER JOIN
    external.gaiaedr3_distance as calj
    ON calj.source_id = gaia.source_id
LEFT OUTER JOIN
    gaiadr3.tmass_psc_xsc_best_neighbour AS tmassxmatch
    ON tmassxmatch.source_id = gaia.source_id
LEFT OUTER JOIN
    gaiadr3.tmass_psc_xsc_join AS tmass_join
    ON tmass_join.clean_tmass_psc_xsc_oid = tmassxmatch.clean_tmass_psc_xsc_oid
LEFT OUTER JOIN
    gaiadr1.tmass_original_valid as tmass
    ON tmass.designation = tmass_join.original_psc_source_id
INNER JOIN
    user_sbuder.gbs_v3 as my_table
    ON my_table.tmass_id = tmass.designation
"""

xmatch_gaia = Gaia.launch_job(query = gaia_query)
xmatch_gaia_results = xmatch_gaia.get_results()
# fix formatting issue for string columns to allow to write to file
for key in xmatch_gaia_results.keys():
    if xmatch_gaia_results[key].dtype == object:
        xmatch_gaia_results[key] = np.array(xmatch_gaia_results[key],dtype=str)
# export as fits file
xmatch_gaia_results.write('individual_crossmatches/gbs_v3_gaiadr3_wo_gamSge-result.fits',overwrite=True)

In [268]:
gamSge_query = """
SELECT
calj.*,
gaia.*
FROM gaiadr3.gaia_source as gaia
LEFT OUTER JOIN
    external.gaiaedr3_distance as calj
    ON calj.source_id = gaia.source_id
WHERE
    gaia.source_id = 1823067317695766784
"""

xmatch_gamSge = Gaia.launch_job(query = gamSge_query)
xmatch_gamSge_results = xmatch_gamSge.get_results()
# fix formatting issue for string columns to allow to write to file
for key in xmatch_gamSge_results.keys():
    if xmatch_gamSge_results[key].dtype == object:
        xmatch_gamSge_results[key] = np.array(xmatch_gamSge_results[key],dtype=str)
# export as fits file
xmatch_gamSge_results['hip_int'] = np.array(['98337'],dtype=int)
xmatch_gamSge_results['tmass_id'] = np.array(['None'],dtype=str)
xmatch_gamSge_results.write('individual_crossmatches/gbs_v3_gaiadr3_gamSge-result.fits',overwrite=True)

In [269]:
# Concatenate gaiadr3 matches
xmatch_gaia_vstack = vstack([xmatch_gaia_results, xmatch_gamSge_results])
xmatch_gaia_vstack.write('individual_crossmatches/gbs_v3_gaiadr3-result.fits',overwrite=True)

### 2.4) Crossmatch with AllWISE

In [270]:
allwise_query = """
SELECT
my_table.hip_int,
allwise.*
FROM gaiadr1.allwise_original_valid as allwise
INNER JOIN
    user_sbuder.gbs_v3_allwise as my_table
    ON my_table.designation = allwise.designation
"""

xmatch_allwise = Gaia.launch_job(query = allwise_query)
xmatch_allwise_results = xmatch_allwise.get_results()
# fix formatting issue for string columns to allow to write to file
for key in xmatch_allwise_results.keys():
    if xmatch_allwise_results[key].dtype == object:
        xmatch_allwise_results[key] = np.array(xmatch_allwise_results[key],dtype=str)
# export as fits file
xmatch_allwise_results.write('individual_crossmatches/gbs_v3_allwise-result.fits',overwrite=True)

### 2.5) Crossmatch with Hipparcos

In [271]:
allwise_query = """
SELECT
vanleeuwen.*
FROM public.hipparcos_newreduction as vanleeuwen
INNER JOIN user_sbuder.gbs_v3 as gbs
    ON gbs.hip_int = vanleeuwen.hip
"""
xmatch_hipparcos = Gaia.launch_job(query = allwise_query)
xmatch_hipparcos_results = xmatch_hipparcos.get_results()
xmatch_hipparcos_results.write('individual_crossmatches/gbs_v3_hipparcos-result.fits',overwrite=True)

# MASTER Table Crossmatch

In [272]:
reload_tables = False

if reload_tables:
    initial_table = Table.read('individual_crossmatches/gbs_v3_tmass_id_original.fits')
    xmatch_hipparcos_results = Table.read('individual_crossmatches/gbs_v3_hipparcos-result.fits')
    xmatch_tmass_results = Table.read('individual_crossmatches/gbs_v3_tmass-result.fits')
    xmatch_gaia_results = Table.read('individual_crossmatches/gbs_v3_gaiadr3-result.fits')
    xmatch_allwise_results = Table.read('individual_crossmatches/gbs_v3_allwise-result.fits')

# Read in all individual tables and rename columns to be unique
xmatch_hipparcos_results['hip_int'] = xmatch_hipparcos_results['hip']
xmatch_hipparcos_results.rename_column('ra','ra_hip')
xmatch_hipparcos_results.rename_column('dec','dec_hip')
xmatch_hipparcos_results.rename_column('plx','parallax_hip')
xmatch_hipparcos_results.rename_column('e_plx','parallax_error_hip')
xmatch_tmass_results.rename_column('ph_qual','ph_qual_tmass')
xmatch_tmass_results.rename_column('DESIGNATION','designation_tmass')
xmatch_gaia_results.rename_column('ra','ra_gaiadr3')
xmatch_gaia_results.rename_column('dec','dec_gaiadr3')
xmatch_gaia_results.rename_column('ra_error','ra_error_gaiadr3')
xmatch_gaia_results.rename_column('dec_error','dec_error_gaiadr3')
xmatch_gaia_results.rename_column('DESIGNATION','designation_gaiadr3')
xmatch_allwise_results.rename_column('DESIGNATION','designation_allwise')
xmatch_allwise_results.rename_column('ra','ra_allwise')
xmatch_allwise_results.rename_column('dec','dec_allwise')
xmatch_allwise_results.rename_column('ph_qual','ph_qual_allwise')

# Add description of origin table to column descriptions
for key in xmatch_hipparcos_results.keys():
    xmatch_hipparcos_results[key].description = 'HIP '+xmatch_hipparcos_results[key].description
for key in xmatch_tmass_results.keys():
    if key == 'ph_qual_tmass': xmatch_tmass_results[key].description = '2MASS Photometric quality flag. Three character flag, one character per band [JHKs], that provides a summary of the net quality of the default photometry in each band. The value for ph_qual is set for a band according to the precedence of the table below. For example, a source that is tested and meets the conditions for category “X” is not tested for subsequent qualities. - “X” - There is a detection at this location, but no valid brightness estimate can be extracted using any algorithm. Default magnitude is null. - “U” - Upper limit on magnitude. Source is not detected in this band, or it is detected, but not resolved in a consistent fashion with other bands. A value of ph_qual=“U” does not necessarily mean that there is no flux detected in this band at the location. - “F” - This category includes sources where a reliable estimate of the photometric error could not be determined. The uncertainties reported for these sources in [jhk]_msigcom are flags and have numeric values >8.0. - “E” - This category includes detections where the goodness-of-fit quality of the profile-fit photometry was very poor, or detections where psf fit photometry did not converge and an aperture magnitude is reported, or detections where the number of frames was too small in relation to the number of frames in which a detection was geometrically possible. - “A” - Detections in any brightness regime where valid measurements were made with [jhk]_snr>10 AND [jhk]_cmsig<0.10857. - “B” - Detections in any brightness regime where valid measurements were made with [jhk]_snr>7 AND [jhk]_cmsig<0.15510. - “C” - Detections in any brightness regime where valid measurements were made with [jhk]_snr>5 AND [jhk]_cmsig<0.21714. - “D” - Detections in any brightness regime where valid measurements were made with no [jhk]_snr or [jhk]_cmsig requirement.'
    elif key == 'designation_tmass': xmatch_tmass_results[key].description = '2MASS Sexagesimal, equatorial position-based source name in the form: hmmssss+ddmmsss[ABC...] The prefix “2MASS J” in not explicitely listed in the designation.'
    elif key == 'tmass_oid': xmatch_tmass_results[key].description = '2MASS Incremental unique numeric identifier (increasing with declination). This is the only field which was not in the original 2MASS catalogue, but was added for cross-match purposes.'
    else:
        xmatch_tmass_results[key].description = '2MASS '+xmatch_tmass_results[key].description
for key in xmatch_gaia_results.keys():
    if key == 'tmass_id': xmatch_gaia_results[key].description = 'Gaia DR3 2MASS identifier used for crossmatch'
    elif key == 'flag': xmatch_gaia_results[key].description = 'Gaia DR3 Flag for distance estimates from Bailer-Jones et al. (2021)'
    elif key == 'designation_gaiadr3': xmatch_gaia_results[key].description = 'Gaia DR3 designation'
    elif key == 'phot_variable_flag': xmatch_gaia_results[key].description = 'Gaia DR3 Photometric variability flag'
    elif key == 'libname_gspphot': xmatch_gaia_results[key].description = 'Gaia DR3 Name of library that achieves the highest mean log-posterior in MCMC samples and was used to derive GSP-Phot parameters in this table '
    else:
        xmatch_gaia_results[key].description = 'GaiaDR3 '+xmatch_gaia_results[key].description
for key in xmatch_allwise_results.keys():
    if key == 'designation_allwise': xmatch_allwise_results[key].description = 'AllWISE designation'
    elif key == 'cc_flags': xmatch_allwise_results[key].description = 'AllWISE Contamination and confusion flag. Four character string, one character per band [W1/W2/W3/W4], that indicates that the photometry and/or position measurements of a source may be contaminated or biased due to proximity to an image artifact. The type of artifact that may contaminate the measurements is denoted by the following codes. Lower-case letters correspond to instances in which the source detection in a band is believed to be real but the measurement may be contaminated by the artifact. Upper-case letters are instances in which the source detection in a band may be a spurious detection of an artifact. - D,d - Diffraction spike. Source may be a spurious detection of (D) or contaminated by (d) a diffraction spike from a nearby bright star on the same image, or - P,p - Persistence. Source may be a spurious detection of (P) or contaminated by (p) a short-term latent image left by a bright source, or - H,h - Halo. Source may be a spurious detection of (H) or contaminated by (h) the scattered light halo surrounding a nearby bright source, or - O,o (letter “o”) - Optical ghost. Source may be a spurious detection of (O) or contaminated by (o) an optical ghost image caused by a nearby bright source, or - 0 (number zero) - Source is unaffected by known artifacts. A source extraction may be affected by more than one type of artifact or condition. In this event, the ccFlags value in each band is set in the following priority order: D,P,H,O,d,p,h,o,0. A source can appear in the AllWISE Source Catalog even if it is flagged as a spurious artifact detection in a band if there is a reliable detection in another band that is not flagged as a spurious artifact detection. CAUTION: Non-zero ccFlags values in any band indicate the the measurement in that band may be contaminated and the photometry should be used with caution.'
    elif key == 'var_flag': xmatch_allwise_results[key].description = 'AllWISE Variability flag. The variability flag is a four-character string, one character per band, in which the value for each band is related to the probability that the source flux measured on the individual WISE exposures was not constant with time. The probability is computed for a band only when there are at least six single-exposure measurements available that satisfy minimum quality criteria. A value of “n” in a band indicates insufficient or inadequate data to make a determination of possible variability. Values of “0” through “9” indicate increasing probabilities of variation. Values of “0” through “5” are most likely not variables. Sources with values of “6” and “7” are likely flux variables, but are the most susceptible to false-positive variability. VarFlag values greater than “7” have the highest probability of being true flux variables in a band. CAUTION: Estimation of flux variability is unreliable for sources that are extended (extFlag>0), and sources whose measurements are contaminated by image artifacts in a band (ccFlags[band] != 0).'
    elif key == 'ph_qual_allwise': xmatch_allwise_results[key].description = 'AllWISE Photometric quality flag. Four character flag, one character per band [W1/W2/W3/W4], that provides a shorthand summary of the quality of the profile-fit photometry measurement in each band, as derived from the measurement signal-to-noise ratio. - A - Source is detected in this band with a flux signal-to-noise ratio >10. - B - Source is detected in this band with a flux signal-to-noise ratio 3'
    else:
        xmatch_allwise_results[key].description = 'AllWISE '+xmatch_allwise_results[key].description

In [273]:
# join init + hipparcos
init_hip = join(initial_table, xmatch_hipparcos_results, keys='hip_int',join_type='left')
# further join with tmass
init_hip_tmass = join(init_hip, xmatch_tmass_results, keys='hip_int',join_type='left')
# further join with gaia
init_hip_tmass_gaia = join(init_hip_tmass, xmatch_gaia_results, keys='hip_int',join_type='left')
# further join with allwise
init_hip_tmass_gaia_allwise = join(init_hip_tmass_gaia, xmatch_allwise_results, keys='hip_int',join_type='left')
init_hip_tmass_gaia_allwise.rename_column('hip_1','hip')
init_hip_tmass_gaia_allwise.rename_column('tmass_id_1','tmass_id')
init_hip_tmass_gaia_allwise.remove_columns(['hip_2','tmass_id_2','source_id_2'])

In [274]:
# description = str(init_hip_tmass_gaia_allwise[key].description)
# description = description.replace('±','-+')
# description = description.replace('∼U','U.T')

# text = description[:800]
# print(text)
# (text).encode('ascii')

In [351]:
init_hip_tmass_gaia_allwise.write('gbs_v3_tmass_gaiadr3_hip_allwise_all_columns_from_fits.fits',overwrite=True)
data,header = fits.getdata('gbs_v3_tmass_gaiadr3_hip_allwise_all_columns_from_fits.fits', header=True)
for each_index, key in enumerate(init_hip_tmass_gaia_allwise.keys()):
    description = str(init_hip_tmass_gaia_allwise[key].description).replace('\n', ' ')
    description = description.replace('±','-+')
    description = description.replace('∼U','U.T')
    description = description.replace('“','"')
    description = description.replace('”','"')
    try:
        header['TCOMM'+str(each_index+1)] = description
    except:
        print(key, description)
    
    if key == 'hip':
        header['TCOMM'+str(each_index+1)] = 'Hipparcos identifier as string, including HIP'
    if key == 'hip_int':
        header['TCOMM'+str(each_index+1)] = 'Hipparcos identifier as integer, excluding HIP'
    if key == 'hd':
        header['TCOMM'+str(each_index+1)] = 'Henry Draper catalog identifier'
    if key == 'ra_j2000':
        header['TCOMM'+str(each_index+1)] = '2MASS Right Ascension, J2000.0'
    if key == 'dec_j2000':
        header['TCOMM'+str(each_index+1)] = '2MASS Declination, J2000.0'
    if key == 'theta_ld':
        header['TCOMM'+str(each_index+1)] = 'Limb-darkened angular diameter'
    if key == 'e_theta_ld':
        header['TCOMM'+str(each_index+1)] = 'Uncertainty of limb-darkened angular diameter'
    if key == 'ref_theta_ld_':
        header['TCOMM'+str(each_index+1)] = 'ADS reference of limb-darkened angular diameter'
    if key == 'teff':
        header['TCOMM'+str(each_index+1)] = 'Effective Temperature from Soubiran et al. (2023)'
    if key == 'e_teff':
        header['TCOMM'+str(each_index+1)] = 'Uncertainty of effective Temperature'
    if key == 'e_teff_percent':
        header['TCOMM'+str(each_index+1)] = 'Relative uncertainty of effective Temperature in percent'
    if key == 'logg':
        header['TCOMM'+str(each_index+1)] = 'Logarithmic surface gravity from Soubiran et al. (2023)'
    if key == 'e_logg':
        header['TCOMM'+str(each_index+1)] = 'Uncertainty of logarithmic surface gravity'
    if key == 'fe_h':
        header['TCOMM'+str(each_index+1)] = 'Logarithmic iron abundance from Soubiran et al. (2023)'
        
    if key in ['fe_h','rv_template_fe_h','mh_gspphot','mh_gspphot_lower','mh_gspphot_upper']:
        header['TUNIT'+str(each_index+1)] = 'dex'
    if key in ['ra_j2000','dec_j2000']:
        header['TUNIT'+str(each_index+1)] = 'deg'
    if key in ['theta_ld','e_theta_ld']:
        header['TUNIT'+str(each_index+1)] = 'mas'
    if key in ['teff','e_teff']:
        header['TUNIT'+str(each_index+1)] = 'K'
    if key in ['logg','e_logg']:
        header['TUNIT'+str(each_index+1)] = 'log(cm.s**-2)'
    if key in ['phot_g_mean_flux','phot_g_mean_flux_error','phot_bp_mean_flux','phot_bp_mean_flux_error','phot_rp_mean_flux','phot_rp_mean_flux_error']:
        header['TUNIT'+str(each_index+1)] = 'electron.s**-1'

In [ ]:
fits_table = fits.BinTableHDU.from_columns(columns=data,header=header)
fits_table.writeto('gbs_v3_tmass_gaiadr3_hip_allwise_all_columns.fits',overwrite=True)